In [1]:
# IMPORTANDO AS BIBLIOTECAS QUE QUE SERÃO UTILIZADAS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import csv
from scipy import stats
from sklearn import metrics

%matplotlib inline

In [ ]:
#----------------------------------------------------------
#1 - IMPORTANDO OS DADOS DE TRABALHO ---TREINO
#----------------------------------------------------------

enem_train = 'train.csv'
enem_test = 'test.csv'

#IMPORTANDO OS DADOS DE TREINO
missing_values = ['n/a', 'na', '--', '.']
df_train = pd.read_csv(enem_train, na_values = missing_values, encoding="UTF8" )

#IMPORTANDO OS DADOS DE TESTE
df_test = pd.read_csv(enem_test, na_values=missing_values, encoding="UTF8" )

In [ ]:
# AMPLIANDO A EXIBIÇÃO DO CAMPO DE SAÍDA - COLUNAS
pd.set_option("display.max_columns", 200)

In [ ]:
# AMPLIANDO A EXIBIÇÃO DO CAMPO DE SAÍDA - LINHAS
pd.set_option("display.max_rows", 15000)

In [ ]:
# 5 PRIMEIRAS LINHAS DO NOSSO CONJ. DE DADOS
df_train.head()

In [ ]:
df_train.info()

In [ ]:
# VERIFICANDO E EXCLUINDO AS FEATURES QUE DO CONJ. DE TREINO QUE NÃO ESTÃO NO CONJ. DE TESTE

for cols in list(df_train):
    if cols not in df_test and cols != 'NU_NOTA_MT':
        df_train.drop(cols, axis = 1, inplace= True)

In [ ]:
# COMO ESTA NOSSO CONJ. DE DADOS?
df_train.shape

In [ ]:
# EXIBINDO AS 5 PRIMEIRAS LINHAS DO NOSSO CONJ. DE DADOS APÓS A ELIMINAÇÃO DE ALGUMAS FEATURES
df_train.head()

In [ ]:
# DAS FEATURES QUE RESTARAM, APÓS UMA ANÁLISE PERCEBEU-SE QUE ALGUMAS SÃO DESNECESSÁRIAS, 
# VAMOS ELIMINA-LAS?

excluCols = ['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'TP_DEPENDENCIA_ADM_ESC', 
             'IN_CEGUEIRA', 'IN_DISLEXIA', 
             'IN_SABATISTA', 'IN_GESTANTE', 
             'IN_IDOSO', 'CO_PROVA_CN', 'CO_PROVA_CH', 
             'CO_PROVA_LC', 'CO_PROVA_MT', 'IN_DISCALCULIA']

df_train.drop(excluCols, axis=1, inplace= True)

In [ ]:
# COMO ESTA NOSSO CONJ. DE DADOS AGORA?
df_train.shape

#### LIMPEZA DOS DADOS FALTANTES

In [ ]:
# VAMOS COMEÇAR AVALIANDO QUAIS COLUNAS DO NOSSO CONJ. DE DADOS APRESENTAM ALGUM VALOR NAN?

df_train.isnull().any()

In [ ]:
# APRESENTADO GRAFICAMENTE AS COLUNAS COM DADOS FALTANTES

msno.bar(df_train.sample(1000));

In [ ]:
# PORCENTAGEM DE VALORES FALTANTES POR FEATURE

(df_train.isnull().sum() / df_train.shape[0]) * 100 

In [ ]:
# ELIMINANDO AS COLUNAS COM PORCENTAGEM ACIMA DE 50% DE VALORES FALTANTES

excluColsNull = ['TP_ENSINO', 'Q027'] 

df_train.drop(excluColsNull, axis = 1, inplace=True )

# As demais não iremos eliminar por que existe uma correlação interessante 
# entre elas e a nossa var Alvo

#### CORRELAÇÃO ENTRE AS VARIÁVEIS

In [ ]:
# POR MEIO DAS CORRELAÇÕES POSITIVAS E MAIS ALTAS, SELECIONAMOS AS FEATURES MAIS RELEVANTES PARA
# O NOSSO MODELO 

# 1º VAMOS VERIFICAR A CORRELAÇÃO DE TODAS AS VARIÁVEIS DO NOSSO CONJUNTO 

corrAlvo1 = sns.heatmap(df_train.corr(method = 'pearson'), annot=True, cmap = 'YlGnBu')

corrAlvo = plt.gcf()
corrAlvo.set_size_inches(22,16)
plt.show()

# PERCEBEMOS QUE ALGUMAS VARIÁVEIS APRESENTAM UM CORRELAÇÃO ALTA E POSISTIVA. VAMOS VÊ-LAS?

In [ ]:
# CORRELAÇÃO ENTRE A NOTA DE MATEMÁTICA E OUTRAS NOTAS

df_train[['NU_NOTA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC', 
          'NU_NOTA_COMP1', 'NU_NOTA_REDACAO', 'NU_NOTA_COMP2', 
          'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']].corr()

In [ ]:
# CORRELAÇÃO ENTRE A NOTA DE MATEMÁTICA E OUTRAS NOTAS - GRAFICAMENTE

corrAlvo1 = sns.heatmap(df_train[['NU_NOTA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC', 
                                  'NU_NOTA_COMP1', 'NU_NOTA_REDACAO', 'NU_NOTA_COMP2', 
                                  'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']]
                                  .corr(method = 'pearson'), annot = True, cmap = 'YlGnBu')
corrAlvo = plt.gcf()
corrAlvo.set_size_inches(8,6)
plt.show()

In [ ]:
# CORRELAÇÃO ENTRE AS FALTAS NAS PROVAS 

df_train[['TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC']].corr()

In [ ]:
# CORRELAÇÃO ENTRE AS PRESENÇAS NAS PROVAS - GRAFICAMENTE

corrAlvo2 = sns.heatmap(df_train[['TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC']]
               .corr(method='pearson'), annot=True, cmap='YlGnBu')
corrAlvo=plt.gcf()
corrAlvo.set_size_inches(8,6)
plt.show()

# Percebemos que existe uma correlação muito alta entre faltas nas provas de outras disciplinas 
# e faltas na prova de matemática. Assim, concluímos que alunos que faltam em outras provas tendem
# a faltar também na prova de matemática.

In [ ]:
# EMBORA A NOTAS DE COMPETENCIA POSSUAM UMA CORRELAÇÃO BOA. A NOTA DE REDAÇÃO É A SOMA DAS 
# COMPETENCIAS 1,2,3,4 e 5, ASSIM, VAMOS EXCLUIR ESSAS INFORMAÇÕES E TRABALHAR APENAS COM A NOTA DA REDAÇÃO

excluColsNull_1 = ['NU_NOTA_COMP1','NU_NOTA_COMP2', 'NU_NOTA_COMP3','NU_NOTA_COMP4', 'NU_NOTA_COMP5']

df_train.drop(excluColsNull_1, axis=1, inplace=True)

In [ ]:
# COMO ESTÁ NOSSO CONJ. DE DADOS AGORA?
df_train.shape

In [ ]:
# APÓS A LIMPEZA NOSSO CONJ DE DADOS PERMANECE COM AS SEGUINTES COLUNAS
df_train.info();

#### ANÁLISE DAS NOTAS DE MATEMÁTICA

##### POR SEXO

In [ ]:
# DISTRIBUIÇÃO DAS NOTAS DE MATEMÁTICA POR SEXO
plt.figure(figsize=(12,4))
plt.title('Distribuição notas de Matemática por Sexo')
sns.distplot(df_train[df_train['TP_SEXO'] == 'M']['NU_NOTA_MT'], kde=True,bins=50, label='Masculino')
sns.distplot(df_train[df_train['TP_SEXO'] == 'F']['NU_NOTA_MT'], kde=True,bins=50, label='Feminino')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(14,4))
ax = sns.catplot(x='TP_SEXO', y='NU_NOTA_MT', data = df_train, kind="box")
plt.show();

In [ ]:
# MÉDIA DAS NOTAS DE MATEMÁTICA POR SEXO
df_train.groupby('TP_SEXO')['NU_NOTA_MT'].mean()

In [ ]:
# DESVIO PADRÃO DAS NOTAS DE MATEMÁTICA POR SEXO
df_train.groupby('TP_SEXO')['NU_NOTA_MT'].std()

In [ ]:
# MEDIANA DAS NOTAS DE MATEMÁTICA POR SEXO
df_train.groupby('TP_SEXO')['NU_NOTA_MT'].median()

##### POR RAÇA

In [ ]:
# Distribuição das notas de matemática por Raça

plt.figure(figsize=(14,6))
plt.title('Distribuição notas de Matemática')
sns.distplot(df_train[df_train['TP_COR_RACA']==0]['NU_NOTA_MT'], kde=True,bins=50, label='Não declarado')
sns.distplot(df_train[df_train['TP_COR_RACA']==1]['NU_NOTA_MT'], kde=True,bins=50, label='Branca')
sns.distplot(df_train[df_train['TP_COR_RACA']==2]['NU_NOTA_MT'], kde=True,bins=50, label='Preta')
sns.distplot(df_train[df_train['TP_COR_RACA']==3]['NU_NOTA_MT'], kde=True,bins=50, label='Parda')
sns.distplot(df_train[df_train['TP_COR_RACA']==4]['NU_NOTA_MT'], kde=True,bins=50, label='Amarela')
sns.distplot(df_train[df_train['TP_COR_RACA']==5]['NU_NOTA_MT'], kde=True,bins=50, label='Indígena ')
sns.distplot(df_train[df_train['TP_COR_RACA']==6]['NU_NOTA_MT'], kde=True,bins=50, label='Não dispõe da informação')
plt.legend()
plt.show();

In [ ]:
ax1 = sns.catplot(x='TP_COR_RACA', y='NU_NOTA_MT', data=df_train, kind='box', aspect=2)
# plt.legend(labels = ['Não declarado', 'Branca', 'Preta', 'Parda', 'Amarela', 'Indígena', 
#                      'Não dispõe da informação'], ncol=7, loc='upper left')
plt.show()



In [ ]:
# MÉDIA DAS NOTAS DE MATEMÁTICA POR RAÇA
df_train.groupby('TP_COR_RACA')['NU_NOTA_MT'].mean()

In [ ]:
# DESVIO PADRÃO DAS NOTAS DE MATEMÁTICA POR RAÇA
df_train.groupby('TP_COR_RACA')['NU_NOTA_MT'].std()

In [ ]:
# MEDIANA DAS NOTAS DE MATEMÁTICA POR RAÇA
df_train.groupby('TP_COR_RACA')['NU_NOTA_MT'].median()

#### ANALIZANDO AS NOTAS DE MATEMÁTICA DOS CANDIDATOS QUE FORAM DESCLASSIFICADOS OU FALTARAM EM OUTRAS PROVAS

##### FALTAS E DESCLASSIFICAÇÃO EM CN

In [ ]:
# NÚMERO DE CANDIDATOS QUE FALTARAM OU FORAM DESCLASSIFICADOS NA PROVA DE Ciências da Natureza

Nota_CN_Missing = df_train[(df_train['NU_NOTA_CN'].isnull() == True)]['NU_NOTA_MT'].shape[0]
Nota_CN_Missing

In [ ]:
# DOS CANDIDATOS, QUANTOS FORAM DESCLASSIFICADOS OU FALTARAM NA PROVA DE MATEMÁTICA?
df_train[(df_train['NU_NOTA_CH'].isnull() == True)&(df_train['NU_NOTA_MT'].isnull() == True)].shape[0]

In [ ]:
# PORCENTAGEM DE CANDIDATOS QUE FORAM DESCLASSIFICADOS OU FALTARAM EM CN, MAS TIVERAM ALGUMA NOTA 
# EM MATEMÁTICAa 

(df_train[(df_train['NU_NOTA_CN'].isnull() == True)&(df_train['NU_NOTA_MT'].isnull() == False)].shape[0]/Nota_CN_Missing)*100

# Observe que apenas 1% das pessoas que foram desclassificadas ou faltaram em Ciências da Natureza, 
# tiveram alguma nota em Matemática

In [ ]:
# PORCENTAGEM DOS CANDIDATOS CLASSIFICADOS ANALIZANDO ESSE QUESITO
(df_train[(df_train['NU_NOTA_CN'].isnull() == True)&(df_train['NU_NOTA_MT'].isnull() == True)].shape[0]/Nota_CN_Missing)*100

In [ ]:
outlier_CN = list(df_train[(df_train['NU_NOTA_CN'].isnull() == True)&(df_train['NU_NOTA_MT'].isnull() == False)].index)

In [ ]:
# QUANTOS SÃO OS OUTLIERS?
len(outlier_CN)

In [ ]:
# QUAIS SÃO ELES?
df_train[(df_train['NU_NOTA_CN'].isnull() == True)&(df_train['NU_NOTA_MT'].isnull() == False)]['NU_NOTA_MT']

In [ ]:
sns.boxplot(df_train[(df_train['NU_NOTA_CN'].isnull() == True)]['NU_NOTA_MT'])

##### FALTAS E DESCLASSIFICAÇÃO EM CH

In [ ]:
# Número de candidatos que faltaram ou foram desclassificadas na prova de Ciências Humanas

Nota_CH_Missing = df_train[(df_train['NU_NOTA_CH'].isnull() == True)]['NU_NOTA_MT'].shape[0]
Nota_CH_Missing

In [ ]:
df_train[(df_train['NU_NOTA_CH'].isnull() == True)&(df_train['NU_NOTA_MT'].isnull() == True)].shape[0]

In [ ]:
# Porcentagem de candidatos que foram desclassificados ou faltaram em CH, mas tiveram alguma 
#nota em Matemática
(df_train[(df_train['NU_NOTA_CH'].isnull() == True)&(df_train['NU_NOTA_MT'].isnull() == False)].shape[0]/Nota_CH_Missing)*100

In [ ]:
# Porcentagem de candidatos que estavam presentes em ambas as provas
(df_train[(df_train['NU_NOTA_CH'].isnull() == True)&(df_train['NU_NOTA_MT'].isnull() == True)].shape[0]/Nota_CH_Missing)*100

In [ ]:
outlier_CH = list(df_train[(df_train['NU_NOTA_CH'].isnull() == True)&(df_train['NU_NOTA_MT'].isnull() == False)].index)

In [ ]:
len(outlier_CH)

In [ ]:
sns.boxplot(df_train[(df_train['NU_NOTA_CH'].isnull() == True)]['NU_NOTA_MT'])

##### FALTAS E DESCLASSIFICAÇÃO EM LC

In [ ]:
# NÚMERO DE PESSOAS QUE FALTARAM OU FORAM DESCLASSIFICADAS NA PROVA DE LINGUAGENS E CÓDIGOS

Nota_LC_Missing = df_train[(df_train['NU_NOTA_LC'].isnull() == True)]['NU_NOTA_MT'].shape[0]
Nota_LC_Missing

In [ ]:
# DOS CANDIDATOS, QUANTOS FORAM DESCLASSIFICADOS OU FALTARAM NA PROVA DE MATEMÁTICA?
df_train[(df_train['NU_NOTA_LC'].isnull() == True)&(df_train['NU_NOTA_MT'].isnull() == True)].shape[0]

In [ ]:
# PORCENTAGEM DE CANDIDATOS QUE FORAM DESCLASSIFICADOS OU FALTARAM EM LC, MAS TIVERAM ALGUMA 
# NOTA EM MATEMÁTICA

(df_train[(df_train['NU_NOTA_LC'].isnull() == True)&(df_train['NU_NOTA_MT'].isnull() == True)].shape[0]/Nota_LC_Missing)*100

#### ELIMINANDO NOTAS DE MATEMÁTICA IGUAL A ZERO

In [ ]:
df_train[(df_train['NU_NOTA_MT']==0)].shape[0]

In [ ]:
notaMT0 = df_train[df_train['NU_NOTA_MT']== 0]['NU_NOTA_MT']
notaMT0

In [ ]:
outlier_MT = list(df_train[(df_train['NU_NOTA_MT']==0)].index)
outlier_MT

#### ELIMINADO OUTLIERS 

In [ ]:
# EXCLUSÃO DE DADOS NOS QUAIS O CANDIDATO FALTOU EM CN, CH OU TEVE NOTA EM MATEMÁTICA IGUAL A ZERO

outliers = list(set(outlier_CN + outlier_CH + outlier_MT))
outliers

In [ ]:
len(outliers)

In [ ]:
df_train= df_train.drop(outliers)

In [ ]:
df_train[(df_train['TP_PRESENCA_CN']==0) &(df_train['NU_NOTA_MT']>600)].shape[0]

In [ ]:
df_train[(df_train['TP_PRESENCA_CH']==0) &(df_train['NU_NOTA_MT']>600)].shape[0]

In [ ]:
outliersCN = list((df_train[(df_train['TP_PRESENCA_CN']==0) &(df_train['NU_NOTA_MT']>600)]).index)
outliersCN

In [ ]:
outliersCH = list((df_train[(df_train['TP_PRESENCA_CH']==0) &(df_train['NU_NOTA_MT']>600)]).index)
outliersCH

In [ ]:
outliersTP = list(set(outliersCN + outliersCH))

In [ ]:
df_train= df_train.drop(outliersTP)

#### VERIFICACO A DISTRIBUIÇÃO DAS NOTAS LEVANDO EM CONSIDERAÇÃO OS VALORES FALTANTES

In [ ]:
sns.distplot(df_train['NU_NOTA_LC'].fillna(0), label='NU_NOTA_LC')
sns.distplot(df_test['NU_NOTA_CH'].fillna(0), label='NU_NOTA_CH')
sns.distplot(df_test['NU_NOTA_LC'].fillna(0), label='NU_NOTA_LC')
sns.distplot(df_test['NU_NOTA_REDACAO'].fillna(0), label='NU_NOTA_REDACAO')
plt.legend()
plt.show()

In [ ]:
# INFERIR ZERO AOS DADOS FALTANTES. PARTINDO DA IDÉIA QUE FALTA IMPLICA EM NOTA ZERO

df_train = df_train.fillna(value={'NU_NOTA_CN':0,'NU_NOTA_CH':0,'NU_NOTA_LC':0,
                                  'NU_NOTA_REDACAO':0, 'TP_STATUS_REDACAO':0, 'NU_NOTA_MT':0})

In [ ]:
# PORCENTAGEM DE VALORES FALTANTES POR FEATURE

(df_train.isnull().sum() / df_train.shape[0]) * 100 

In [ ]:
df_train.columns.values

In [ ]:
df_train.shape

In [ ]:
# TRANSFOAMNDO DADOS DE TEXTUAIS EM NÚMERICOS

from sklearn.preprocessing import LabelEncoder

var_cat1 = ['SG_UF_RESIDENCIA', 'TP_SEXO', 'TP_COR_RACA',
       'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO','TP_ESCOLA',
       'IN_TREINEIRO', 'IN_BAIXA_VISAO', 'IN_SURDEZ','TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_LINGUA', 'TP_STATUS_REDACAO','Q001','Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']

le = LabelEncoder()

for i in var_cat1: 
    df_train[i] = le.fit_transform(df_train[i].astype(str))
    df_train.dtypes

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
# DUMMIES DE VARIÁVEIS CATEGÓRICAS

var_cat2 = ['SG_UF_RESIDENCIA', 'TP_SEXO', 'TP_COR_RACA',
       'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO','TP_ESCOLA',
       'IN_TREINEIRO', 'IN_BAIXA_VISAO', 'IN_SURDEZ','TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_LINGUA', 'TP_STATUS_REDACAO','Q025', 'Q026', 'Q047']

df_train = pd.get_dummies(df_train, columns = var_cat2).copy()

In [ ]:
df_train.shape

In [ ]:
df_train.head()

#### CONJ. DE DADOS DE TESTE

In [ ]:
df_test.shape

In [ ]:
df_test.head()

In [ ]:
df_test.columns

In [ ]:
# EXCLUINDO COLUNAS IRRELEVANTES

colsExclud_test = ['CO_UF_RESIDENCIA', 'TP_DEPENDENCIA_ADM_ESC', 
                   'IN_CEGUEIRA', 'IN_DISLEXIA', 'IN_SABATISTA', 'IN_GESTANTE', 
                   'IN_IDOSO', 'CO_PROVA_CN', 'CO_PROVA_CH', 
                   'CO_PROVA_LC', 'CO_PROVA_MT', 'IN_DISCALCULIA']

# Removendo as colunas analisadas do conj. de teste
df_test.drop(colsExclud_test, axis=1, inplace=True)

In [ ]:
df_test.shape

In [ ]:
df_test.shape

In [ ]:
# EMBORA A NOTAS DE COMPETENCIA POSSUAM UMA CORRELAÇÃO BOA. A NOTA DE REDAÇÃO É A SOMA DAS 
# COMPETENCIAS 1,2,3,4 e 5, ASSIM, VAMOS EXCLUIR ESSAS INFORMAÇÕES E TRABALHAR APENAS COM A NOTA DA REDAÇÃO

excluColsNull_t1 = ['NU_NOTA_COMP1','NU_NOTA_COMP2', 'NU_NOTA_COMP3','NU_NOTA_COMP4', 'NU_NOTA_COMP5']

df_test.drop(excluColsNull_t1, axis=1, inplace=True)

In [ ]:
df_test.shape

In [ ]:
# CALCULANDO A PORCENTAGEM DE VALORES MISSING NO RESTANTE DO CONJ. DE DADOS

(df_test.isnull().sum() / df_test.shape[0]) * 100

In [ ]:
# ELIMINANDO AS COLUNAS COM PORCENTAGEM ACIMA DE 50% DE VALORES FALTANTES

excluColsNull_t = ['TP_ENSINO', 'Q027'] 

df_test.drop(excluColsNull_t, axis=1, inplace=True)

In [ ]:
df_test.shape

In [ ]:
df_test = df_test.fillna(value={'NU_NOTA_CN':0,'NU_NOTA_CH':0,'NU_NOTA_LC':0,
                                'NU_NOTA_REDACAO':0, 'TP_STATUS_REDACAO':0})

In [ ]:
# Calculando a porcentagem de valores MIssing no restante do Conj. de dados
(df_test.isnull().sum() / df_test.shape[0]) * 100 

In [ ]:
# TRANSFORMANDO DADOS DE TEXTUAIS EM NUMÉRICOS

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in var_cat1: 
    df_test[i] = le.fit_transform(df_test[i].astype(str))
    df_test.dtypes

In [ ]:
df_test.head()

In [ ]:
cat_features_t = ['SG_UF_RESIDENCIA', 'TP_SEXO', 'TP_COR_RACA',
                  'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO','TP_ESCOLA',
                  'IN_TREINEIRO', 'IN_BAIXA_VISAO', 'IN_SURDEZ','TP_PRESENCA_CN', 
                  'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_LINGUA', 
                  'TP_STATUS_REDACAO','Q025', 'Q026', 'Q047']

In [ ]:
df_test1 = pd.get_dummies(df_test, columns = cat_features_t).copy()

In [ ]:
df_test1.head()

In [ ]:
colunas_df_t1 = list(df_test1.columns)
colunas_df2 = list(df_train.columns)

for i in colunas_df2:
    if i not in colunas_df_t1:
        print(i)

In [ ]:
df_train=df_train.drop(columns=['TP_PRESENCA_CN_2','TP_PRESENCA_CH_2', 'TP_ESCOLA_3'])

In [ ]:
colsExclud_3 =[ 'SG_UF_RESIDENCIA_0', 'TP_SEXO_0','TP_COR_RACA_0','TP_NACIONALIDADE_0','TP_ST_CONCLUSAO_3',
         'IN_TREINEIRO_0','IN_BAIXA_VISAO_0','IN_SURDEZ_0','TP_PRESENCA_CN_0',
         'TP_PRESENCA_CH_0','TP_PRESENCA_LC_0','TP_LINGUA_1','TP_STATUS_REDACAO_0','Q025_0', 'Q026_0']

df_train_2 = df_train.drop(columns=colsExclud_3)
df_test1 = df_test1.drop(columns=colsExclud_3)

In [ ]:
df_train_2.columns.values

#### MODELOS PREDITIVOS

In [ ]:
# DIVIDINDO O NOSSO CONJUNTO EM TREINO E TESTE

X = df_train_2.drop(columns =['NU_NOTA_MT'], axis=1)
y = df_train_2['NU_NOTA_MT']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

# NORMALIZANDO OS DADOS
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()  
# X_train = sc.fit_transform(X_train)  
# X_test = sc.transform(X_test)

#### MODELO RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_lr = RandomForestRegressor( 
           criterion='mae', 
           max_depth=8,
           max_leaf_nodes=None,
           min_impurity_split=None,
           min_samples_leaf=1,
           min_samples_split=2,
           min_weight_fraction_leaf=0.0,
           n_estimators= 500,
           n_jobs=-1,
           random_state=0,
           verbose=0,
           warm_start=False
)

model_lr.fit(X_train, y_train)
#y_pred_lr = model_lr.predict(X_test)

In [ ]:
y_pred_test = model_lr.predict(X_test)
y_pred_train = model_lr.predict(X_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mean_squared_error(y_test, y_pred_test)

#### MÉTRICAS DE AVALIAÇÃO DO MODELO

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_train, y_pred_train).round(8)  )
print('MSE:', metrics.mean_squared_error(y_train, y_pred_train).round(8) )  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)).round(8))

In [ ]:
X_testN = df_test1.drop(columns =['NU_INSCRICAO'], axis=1)

ypredict_test = model_lr.predict(X_testN)

In [ ]:
df_test1['NU_NOTA_MT'] = ypredict_test

In [ ]:
answer = df_test1[['NU_INSCRICAO','NU_NOTA_MT']]

In [ ]:
# SALVANDO NOSSA RESPOSTA
answer.to_csv("answer.csv",index=False)